In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os

# modify the project path below accordingly
PATH = "/content/drive/MyDrive/HGR/gesture_face_features"

sys.path.append(os.path.join(PATH, "code", "main"))

data_path = os.path.join(PATH, 'samples', 'sign')
feat_path = os.path.join(PATH, 'data', 'feats')

%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.utils.data import DataLoader

import numpy as np

In [18]:
# loading modules from system path
from loader import SignImageFolder, ModelLoader
from participant import participants, filter_participants, filter_targets
from feature import extract_features, extract_model_features, filter_by_id

In [5]:
dataset = SignImageFolder(data_path)
dataloader = DataLoader(dataset, batch_size=32, num_workers=2)
# have separate dataloader for each model based on input_size
# and recalculate features with extract_features()

In [6]:
feats = extract_model_features(dataloader, feat_path=feat_path)

Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/vgg19_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/resnet50_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/inception_v3_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b0_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b1_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b6_feats.pt.


# Model Transferability Estimation

## LogME

In [7]:
!git clone https://github.com/thuml/LogME

fatal: destination path 'LogME' already exists and is not an empty directory.


In [8]:
from LogME.LogME import LogME
logme = LogME(regression=False)

In [9]:
for model_name in ModelLoader.available_models:
  # feats keys are ...conv, gap, fc
  fc_layer = list(feats[model_name].keys())[-1]
  feat_tensor = feats[model_name][fc_layer]
  f = feat_tensor.cpu() if feat_tensor.is_cuda else feat_tensor.numpy()
  y = np.array(dataset.targets)
  score = logme.fit(f,y)
  print(f'LogME score of {model_name}: {score}')

LogME score of vgg19: 13.308188385500753
LogME score of resnet50: 7.032196750365981


/content/LogME/LogME.py:172: UserWarning: re-fitting for new data. old parameters cleared.
  warnings.warn('re-fitting for new data. old parameters cleared.')


LogME score of inception_v3: 10.980468061676675
LogME score of efficientnet_b0: 10.556121201640433
LogME score of efficientnet_b1: 10.407252203493226
LogME score of efficientnet_b6: 11.352897218778145


In [25]:
# makes copies of feats and uses a lots of RAM
p = filter_participants(feats, dataset)
p_targets = filter_targets(dataset)

In [26]:
for i in range(6):
  for model_name in ModelLoader.available_models:
    # feats keys are ...conv, gap, fc
    fc_layer = list(p[i][model_name].keys())[-1]
    feat_tensor = p[i][model_name][fc_layer]
    f = feat_tensor.cpu() if feat_tensor.is_cuda else feat_tensor.numpy()
    y = p_targets[i]
    score = logme.fit(f, y)
    print(f'Participant {i} LogME score of {model_name}: {score}')
  print()

/content/LogME/LogME.py:172: UserWarning: re-fitting for new data. old parameters cleared.
  warnings.warn('re-fitting for new data. old parameters cleared.')


Participant 0 LogME score of vgg19: 103.50125179359395
Participant 0 LogME score of resnet50: 49.53502664058891
Participant 0 LogME score of inception_v3: 70.5303866697611
Participant 0 LogME score of efficientnet_b0: 70.39628327893641
Participant 0 LogME score of efficientnet_b1: 65.26026329383014
Participant 0 LogME score of efficientnet_b6: 81.78603751879086

Participant 1 LogME score of vgg19: 96.84919190296019
Participant 1 LogME score of resnet50: 51.2964902863794
Participant 1 LogME score of inception_v3: 79.97703405853551
Participant 1 LogME score of efficientnet_b0: 78.20622879553682
Participant 1 LogME score of efficientnet_b1: 69.55740690590521
Participant 1 LogME score of efficientnet_b6: 86.70922599361597

Participant 2 LogME score of vgg19: 99.16203326764625
Participant 2 LogME score of resnet50: 55.83447861696049
Participant 2 LogME score of inception_v3: 77.55328813010593
Participant 2 LogME score of efficientnet_b0: 75.706535784587
Participant 2 LogME score of efficien